In [33]:
# Import the Data
import json
from pprint import pprint

#Read in submissions
with open('r_data/RS_2014-12_speedrun.json', 'r', encoding='utf-8') as file:
    submission_dataset = json.loads(file.read())

# Read in comments
with open('r_data/RC_2014-12_speedrun.json', 'r', encoding='utf-8') as file:
    comment_dataset = json.loads(file.read())

file.close()
#pprint(type(submission_dataset))
pprint(submission_dataset[0])
#pprint(comment_dataset[0])

{'archived': True,
 'author': 'intheblue',
 'author_flair_css_class': '',
 'author_flair_text': 'Hotline Miami, Spider Solitare',
 'created': 1417395407,
 'created_utc': '1417395407',
 'distinguished': None,
 'domain': 'twitch.tv',
 'downs': 0,
 'edited': False,
 'from': None,
 'from_id': None,
 'from_kind': None,
 'gilded': 0,
 'hide_score': False,
 'id': '2nw69z',
 'is_self': False,
 'link_flair_css_class': None,
 'link_flair_text': None,
 'media': {'oembed': {'description': 'Recorded live on an hour ago',
                      'height': 300,
                      'html': '&lt;iframe class="embedly-embed" '
                              'src="//cdn.embedly.com/widgets/media.html?src=http%3A%2F%2Fwww.twitch.tv%2Fwidgets%2Flive_embed_player.swf%3Fchannel%3Ddingodroles&amp;fv=hostname%3Dwww.twitch.tv%26start_volume%3D25%26channel%3Ddingodrole%26auto_play%3Dfalse&amp;url=http%3A%2F%2Fwww.twitch.tv%2Fdingodrole%2Fc%2F5593916&amp;image=http%3A%2F%2Fstatic-cdn.jtvnw.net%2Fjtv_user_pictures%

In [32]:
# Data preporcessing part 1
# We curate the tags we want.       

# process submission
info = {}
submission_dataset_two = []
for dictionary in submission_dataset:
    info = {
        "id":            dictionary.get('id'),
        "title":         dictionary.get('title'),
        "selftext":      dictionary.get('selftext'),
        "score":         dictionary.get('score'),
        "num_comments":  dictionary.get('num_comments')
    }
    submission_dataset_two.append(info)
print("Submission 0:  ")
print(submission_dataset_two[0])
print("\n")

info_two = {}
comment_dataset_two = []
for dictionary in comment_dataset:
    info_two = {
        "parent_id": dictionary.get('parent_id'),
        "body":      dictionary.get('body'),
        "score":     dictionary.get('score')
    }
    comment_dataset_two.append(info_two)
print("Comment 0:  ")
print(comment_dataset_two[0])

with open('r_data/sub_data.json', 'w', encoding='utf-8') as file:
    json.dump(submission_dataset_two, file, ensure_ascii=False, indent=4)

with open('r_data/com_data.json', 'w', encoding='utf-8') as file:
    json.dump(comment_dataset_two, file, ensure_ascii=False, indent=4)
    
file.close()

Submission 0:  
{'id': '2nw69z', 'title': '[WR]Hotline Miami Any% NG in 20:16 by Dingodrole', 'selftext': '', 'score': 37, 'num_comments': 6}


Comment 0:  
{'parent_id': 't3_2nvw48', 'body': 'this only beats my PB time by around a minute :/ a TAS of this should be pretty close to 48:30 easily...', 'score': 6}


In [1]:
# Data preporcessing part 2
# We need to remove anything that has dead data -- like removed, unreconizable words
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
# Topic model LDA
# First attempt is to use Sub title + self text to generate topics
# Second attempt is to use just sub title + comments
# third attempt is just look at comments
# fourth is title + self text + comments
#Read in curated submissions
# Code by ... from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for Gensim model
# Code by ... from: https://stackabuse.com/python-for-nlp-topic-modeling/ for LDA model

# import re
# import numpy as np
# import pandas as pd
# from pprint import pprint

# # Gensim
# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel

# # spacy for lemmatization
# import spacy

# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

# # Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# # Handle the NLTK stopwords
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


import pandas as pd
import numpy as np

submission_dataset_three = pd.read_json(r'r_data/sub_data.json')

# shows the data
# submission_dataset_three.dropna()

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(submission_dataset_three['title'].values.astype('U'))

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

# # open the submissions file
# with open('r_data/sub_data.json', 'r', encoding='utf-8') as file:
#     submission_dataset_three = json.loads(file.read())


#Read in curated comments
# with open('r_data/com_data.json', 'r', encoding='utf-8') as file:
#     comment_dataset_three = json.loads(file.read())



changes
kazooie
good
evil
legend
thc98
36
retro
streamers
planning


In [19]:
# Print words -- manually look at topics
first_topic = LDA.components_[0]
top_topic_words = first_topic.argsort()[-10:]

# for i in top_topic_words:
#     print(count_vect.get_feature_names()[i])

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')



need
island
13
super
help
46
mario
games
speedrun
wr
Top 10 words for topic #0:
['need', 'island', '13', 'super', 'help', '46', 'mario', 'games', 'speedrun', 'wr']


Top 10 words for topic #1:
['jodenstone', 'weekly', 'thread', 'speed', 'oot', 'time', '18', 'december', 'run', 'wr']


Top 10 words for topic #2:
['28', 'speed', '10', 'igt', 'souls', 'marathon', 'dark', '12', '50', 'wr']


Top 10 words for topic #3:
['36', 'race', 'pb', '28', '24', '33', '64', 'rta', '21', 'wr']


Top 10 words for topic #4:
['hawk', '100', 'mario', 'race', 'super', 'game', 'agdq', 'speedrunning', 'new', 'speedrun']




In [20]:
# probability of all topics to each document
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(412, 5)

In [21]:
# see the topic index with max value
submission_dataset_three['Topic'] = topic_values.argmax(axis=1)
submission_dataset_three.head()

,id,title,selftext,score,num_comments,Topic
0,2nw69z,[WR]Hotline Miami Any% NG in 20:16 by Dingodrole,,37,6,3
1,2nwjc2,Couple questions about Chrono Trigger Any%,1) Where can I find more information on the rn...,3,6,4
2,2nwl8q,[WR] Luigi's Mansion 100% new WR by Veman300! ...,,13,2,1
3,2nwwf8,Jak and Daxter any% route?,Does anyone know of a route that is written ou...,1,2,0
4,2nxll5,[WR] Castlevania:Harmony Of Dissonance Maxim A...,,16,6,3
